In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import re
from collections import Counter
from tqdm import tqdm

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config

import ilm.ilm.tokenize_util
from ilm.ilm.infer import infill_with_ilm
from perturbation_functions import calculate_necc_and_suff, gen_num_samples_table, gen_probs_table

/Users/joanna/Documents/mva/S2/NLP/project_nlp/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_DIR = 'Models/ILM/'
MASK_CLS = 'ilm.mask.hierarchical.MaskHierarchical'

tokenizer = ilm.ilm.tokenize_util.Tokenizer.GPT2
with open(os.path.join(MODEL_DIR, 'additional_ids_to_tokens.pkl'), 'rb') as f:
    additional_ids_to_tokens = pickle.load(f)
additional_tokens_to_ids = {v:k for k, v in additional_ids_to_tokens.items()}
try:
    ilm.ilm.tokenize_util.update_tokenizer(additional_ids_to_tokens, tokenizer)
except ValueError:
    print('Already updated')
print(additional_tokens_to_ids)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = GPT2LMHeadModel.from_pretrained(MODEL_DIR)
config = GPT2Config.from_json_file(f'{MODEL_DIR}config.json')
model = GPT2LMHeadModel(config)
# Charger les poids
model.load_state_dict(torch.load(f'{MODEL_DIR}model_weights.pt', map_location=torch.device('cpu')))
model.eval()
_ = model.to(device)

{'<|startofinfill|>': 50257, '<|endofinfill|>': 50258, '<|infill_document|>': 50259, '<|infill_paragraph|>': 50260, '<|infill_sentence|>': 50261, '<|infill_ngram|>': 50262, '<|infill_word|>': 50263}


In [3]:
# generate approximately 100 perturbations for each token. 
num_samples = gen_num_samples_table(50, 100)
probs_table = gen_probs_table(50)
mask_tokn = additional_tokens_to_ids['<|infill_ngram|>']

orig_texts = []
necc_perturbed = []
suff_perturbed = []
necc_masks = []
suff_masks = []

with open("extension_sexism/data_sexism/samples_sexism.txt", "r") as ff:
    with tqdm(total=120) as pbar:
        for text in ff:
            necc_pp, suff_pp, necc_mm, suff_mm = calculate_necc_and_suff(text, ilm_tokenizer=tokenizer, ilm_model=model, cl_tokenizer=None, cl_model=None, num_samples=num_samples,
                               mask_tokn=mask_tokn, additional_tokens_to_ids=additional_tokens_to_ids, probs_table=probs_table, 
                               return_pert_only=True)

            orig_texts.append(text)
            necc_perturbed.append(necc_pp)
            suff_perturbed.append(suff_pp)
            necc_masks.append(necc_mm)
            suff_masks.append(suff_mm)
            pbar.update(1)
    
necc_suff_perturbations = {'orig_texts': orig_texts, 
                           'necc_perturbed': necc_perturbed, 
                           'suff_perturbed': suff_perturbed,
                           'necc_masks': necc_masks,
                           'suff_masks': suff_masks}

pickle.dump(necc_suff_perturbations, open('extension_sexism/data_sexism/sexism_necc_suff_perturbations.pickle', 'wb'))

100%|██████████| 120/120 [45:17:39<00:00, 1358.83s/it]  
